# Café Sales Data Analysis - Business Intelligence Report

## Executive Summary
This notebook provides a comprehensive analysis of café sales data to help the café owner make data-driven decisions for marketing campaigns and operational improvements. We analyze 10,000 transactions from 2023 to identify peak business periods, customer preferences, and revenue optimization opportunities.

## Business Objectives
1. **Analyze popular selling times**: 
- When are the busiest AND most profitable times?
- Are weekends busier than weekdays?
2. **Understand Customer Behavior**: What items do customers spend most on?
3. **Optimize Operations**: Which payment methods and locations drive the most revenue?
4. **Data Quality Assessment**: What issues exist in our transaction data?

---

1. Data Import, Cleaning and Preparation

In [17]:
#Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [18]:
#The na_values replaces all the unknown and error values to NaN
df = pd.read_csv('/Users/kabbo/Desktop/marcy/Project 2 Cafe Sales/M1-FINAL-PROJECT-THIERNO_KABBO/data/raw/cafe_sales.csv', na_values=["UNKNOWN", "ERROR", ""])


#df = pd.read_csv('/Users/kabbo/Desktop/marcy/Project 2 Cafe Sales/M1-FINAL-PROJECT-THIERNO_KABBO/data/raw/cafe_sales.csv')


# Look at first few rows
print("\nFirst 5 rows:")
print(df["Transaction Date"].value_counts())

# Get basic information
print(f"Dataset shape: {df.shape}")  # (rows, columns)
print(f"Total transactions: {df.shape[0]:,}")


First 5 rows:
Transaction Date
2023-02-06    40
2023-06-16    40
2023-03-13    39
2023-07-21    39
2023-09-21    39
              ..
2023-04-27    15
2023-11-24    15
2023-03-11    14
2023-02-17    14
2023-07-22    14
Name: count, Length: 365, dtype: int64
Dataset shape: (10000, 8)
Total transactions: 10,000


In [19]:
df['Item'] = df['Item'].fillna('UNKNOWN')

#print(df['Item'].isnull().sum())  # Should print 0
#print(df['Item'].unique())        # Should include 'UNKNOWN' in place of NaN before
print(df['Item'].value_counts())

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      969
Name: count, dtype: int64


In [20]:
# Get detailed information about each column
print("Data types and missing values:")
df.info()


# Check data types
print("\nColumn data types:")
for col in df.columns:
    print(f"  • {col}: {df[col].dtype}")




Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              10000 non-null  object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9467 non-null   float64
 4   Total Spent       9498 non-null   float64
 5   Payment Method    6822 non-null   object 
 6   Location          6039 non-null   object 
 7   Transaction Date  9540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB

Column data types:
  • Transaction ID: object
  • Item: object
  • Quantity: float64
  • Price Per Unit: float64
  • Total Spent: float64
  • Payment Method: object
  • Location: object
  • Transaction Date: object


In [21]:
# Count unique values in each column
print("\nUnique values per column:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"  • {col}: {unique_count:,} unique values")




Unique values per column:
  • Transaction ID: 10,000 unique values
  • Item: 9 unique values
  • Quantity: 5 unique values
  • Price Per Unit: 6 unique values
  • Total Spent: 17 unique values
  • Payment Method: 3 unique values
  • Location: 2 unique values
  • Transaction Date: 365 unique values


In [22]:
# Check for missing values
print("Missing values analysis:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent.round(2)
})
print(missing_summary)

Missing values analysis:
                  Missing Count  Missing Percentage
Transaction ID                0                0.00
Item                          0                0.00
Quantity                    479                4.79
Price Per Unit              533                5.33
Total Spent                 502                5.02
Payment Method             3178               31.78
Location                   3961               39.61
Transaction Date            460                4.60


In [23]:
#Checking for duplicates

duplicate_rows = df[df.duplicated(subset=['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Transaction Date'], keep=False)]
print("Duplicate Rows:")
print(duplicate_rows)

num_duplicates = df.duplicated(subset=['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Transaction Date'], keep=False).sum()
print(f"Number of duplicate rows: {num_duplicates}")

Duplicate Rows:
Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []
Number of duplicate rows: 0


In [24]:
# Imputing each column depending on skew results that we got from Excel (Thierno will explain this)

# We will use Average for Quantity column
# Calculate the mean of the 'Quantity' column, ignoring NaN values
quantity_median = df['Quantity'].median()
# Impute the NaN values in 'Quantity' with the calculated mean
df['Quantity'].fillna(quantity_median, inplace=True)

df['Quantity'] = df['Quantity'].astype(int)
#print(df['Quantity'].head())

print((df['Quantity'].sum()))  # Prints 0 which means no erros or unknowns
#print(quantity_median)



30271


/var/folders/78/8f22v_ln4kvg9b_7fr3_2nfh0000gn/T/ipykernel_33765/2137753788.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Quantity'].fillna(quantity_median, inplace=True)


In [25]:
# There is no skew on the Price Per Unit column so we will just use average
pricer_per_unit_mean = df['Price Per Unit'].mean()

df['Price Per Unit'].fillna(pricer_per_unit_mean, inplace=True)

#print((df['Price Per Unit'].mean())) 

#print((df['Price Per Unit'].sum())) 
#print(df['Price Per Unit'].isnull().sum())  # Prints 0 which means no erros or unknowns


/var/folders/78/8f22v_ln4kvg9b_7fr3_2nfh0000gn/T/ipykernel_33765/3480734360.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price Per Unit'].fillna(pricer_per_unit_mean, inplace=True)


In [26]:
# For the Total Spent column, we will multiply ppu with quantity

df['Total Spent'] = df['Price Per Unit'] * df['Quantity']


#print((df['Total Spent'].sum())) 

print(df.head(10))


  Transaction ID      Item  Quantity  Price Per Unit  Total Spent  \
0    TXN_1961373    Coffee         2             2.0          4.0   
1    TXN_4977031      Cake         4             3.0         12.0   
2    TXN_4271903    Cookie         4             1.0          4.0   
3    TXN_7034554     Salad         2             5.0         10.0   
4    TXN_3160411    Coffee         2             2.0          4.0   
5    TXN_2602893  Smoothie         5             4.0         20.0   
6    TXN_4433211   UNKNOWN         3             3.0          9.0   
7    TXN_6699534  Sandwich         4             4.0         16.0   
8    TXN_4717867   UNKNOWN         5             3.0         15.0   
9    TXN_2064365  Sandwich         5             4.0         20.0   

   Payment Method  Location Transaction Date  
0     Credit Card  Takeaway       2023-09-08  
1            Cash  In-store       2023-05-16  
2     Credit Card  In-store       2023-07-19  
3             NaN       NaN       2023-04-27  
4  Di

In [27]:
# For Items column, leave it as it is
#items_mode = df['Item'].mode()

#df['Item'].fillna(items_mode, inplace=True)

#print(items_mode)
#print(df['Item'].items_mode.sum()) # Prints 0 which means no erros or unknowns


In [28]:

#Payment Method leave it alone

#Location leave it alone




In [ ]:
#Transaction Date we will drop all unknown
df = df.dropna(subset=['Transaction Date'])

print(df['Transaction Date'].isnull().sum())  # Should print 0
print(df['Transaction Date'].count())        # See all unique values




0
9540


In [30]:
df.to_csv('cleaned_cafe_sales.csv', index=False)